# Home work week 1 Jarosław Kulesza

## Imports

In [1]:
from __future__ import print_function

In [2]:
import json

import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

## Task 1

In [3]:
data_frame_csv = pd.read_csv("./yelp.csv")[["stars", "text"]]



In [8]:
from io import StringIO

data_json = []
with open('./yelp.json') as data_file:
    for line in data_file.readlines():
        obj = json.load(StringIO(line))
        data_json.append((obj['stars'], obj['text']))
        
data_frame_json = pd.DataFrame(data_json, columns=["stars", "text"])

In [9]:
print(data_frame_csv.shape)
print(data_frame_json.shape)

(10000, 2)
(10000, 2)


In [10]:
#data = data_frame_csv

# optional
data = data_frame_json

## Task 2

In [11]:
filtered_data = data[(data.stars==1)|(data.stars==5)]

In [12]:
filtered_data.head()

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...
6,5,Drop what you're doing and drive here. After I...


## Task 3

In [13]:
X = filtered_data.text

In [14]:
y = filtered_data.stars

In [15]:
X.shape


(4086,)

In [16]:
y.shape

(4086,)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)

(3064,)
(1022,)


## Task 4

In [18]:
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

## Task 5

In [19]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
y_pred_class = nb.predict(X_test_dtm)

In [21]:
metrics.accuracy_score(y_test, y_pred_class)


0.91878669275929548

In [22]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[126,  58],
       [ 25, 813]])

## Task 6

In [23]:
y_test.value_counts().head(1) / len(y_test)

5    0.819961
dtype: float64

## Task 7

In [24]:
nb.feature_count_

array([[ 26.,   4.,   1., ...,   0.,   0.,   0.],
       [ 39.,   5.,   0., ...,   1.,   1.,   1.]])

In [25]:
X_train_tokens = vect.get_feature_names()

In [26]:
token_counts = pd.DataFrame({'token':X_train_tokens, 'one':nb.feature_count_[0, :], 'five':nb.feature_count_[1, :]})

In [27]:
token_counts['one'] = token_counts.one + 1
token_counts['five'] = token_counts.five + 1

In [28]:
token_counts['one_ratio'] = token_counts.one / token_counts.five
token_counts['five_ratio'] = token_counts.five / token_counts.one

In [29]:
# most predictive tokens of 1 star
token_counts.sort('one_ratio', ascending=False).head()

,five,one,token,one_ratio,five_ratio
14110,1,17,staffperson,17,0.058824
12172,1,14,refused,14,0.071429
4538,2,24,disgusting,12,0.083333
5780,1,11,filthy,11,0.090909
15649,1,9,unacceptable,9,0.111111


In [30]:
# most predictive tokens of 5 star
token_counts.sort('five_ratio', ascending=False).head()

,five,one,token,one_ratio,five_ratio
5596,193,2,fantastic,0.010363,96.500000
10949,245,3,perfect,0.012245,81.666667
16751,62,1,yum,0.016129,62.000000
5643,345,7,favorite,0.020290,49.285714
10527,49,1,outstanding,0.020408,49.000000


## Task 8

In [31]:
for doc in X_test[y_test < y_pred_class][:5]:
    print(doc)
    print("--------------")

This has to be the worst restaurant in terms of hygiene. Two of my friends had food -poisoning after having dinner here. The food is just unhealthy with tons of oil floating on the top of curries, and I am not sure if any health/hygiene code is followed here. 
The service is poor and the information on its website is incorrect, the owner does not allow dine-in after 9 or 10 even though it says that the restaurant is open till 11. 

One night I saw the owner cleaning the place without gloves and she was nice enough to give us a to-go parcel without cleaning her hands (great example to the servers!). I had a peek inside the kitchen when the door was ajar, and it definitely looked dirty.

I have been a lot of hole-in-the-wall places around this restaurant, including Haji Baba, the Vietnamese place and others, but neither any of my friends nor I have fallen sick coz of the food. If you need a spicy-food fix, i strongly recommend you do not try this place, lest you want a visit to the docto

In [32]:
for doc in X_test[y_test > y_pred_class][:5]:
    print(doc)
    print("--------------")

I now consider myself an Arizonian. If you drive a lot on the 101 or 51 like I do, you'll get your fair share of chips on your windshield. You'll also have to replace a windshield like I had to do just recently. Apparently, chips and cracking windshields  is common in Arizona. In fact, I seem to recall my insurance agent telling me that insurance companies must provide this coverage in Arizona.

I had a chip repaired about a year ago near the very bottom of the windshield. Just recently a small, very fine crack started traveling north on the windshield from the repaired chip (a different vendor repaired the chip). I called these guys over to my house and they said it was too long to fix, so they replaced the whole windshield the next day.

What great service, they come out to your residence or place of business to repair or replace your windshield.
--------------
This is by far my favourite department store, hands down. I have had nothing but perfect experiences in this store, without 

In [33]:
# I think in this exaples are many words with opposite meaning. So in posts with one star are many words like: like, very, fine, good.
# In posts with five stars are many words with negative meaning: never, broke, vandalized.
# And machine had problems with irony etc.

## Task 9

In [34]:
X_all = data.text
y_all = data.stars

In [35]:
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y_all, random_state=1)

In [36]:
vect_all = CountVectorizer()
X_train_all_dtm = vect_all.fit_transform(X_train_all)
X_test_all_dtm = vect_all.transform(X_test_all)

In [37]:
nb_all = MultinomialNB()
nb_all.fit(X_train_all_dtm, y_train_all)
y_pred_class_all = nb_all.predict(X_test_all_dtm)

In [38]:
metrics.accuracy_score(y_test_all, y_pred_class_all)

0.47120000000000001

In [39]:
y_test_all.value_counts().head(1) / len(y_test_all)

4    0.3536
dtype: float64

In [40]:
metrics.confusion_matrix(y_test_all, y_pred_class_all)

array([[ 55,  14,  24,  65,  27],
       [ 28,  16,  41, 122,  27],
       [  5,   7,  35, 281,  37],
       [  7,   0,  16, 629, 232],
       [  6,   4,   6, 373, 443]])

Comment: NaiveBayes had problems with classification all star ratings (only 47% accuracy). System is very confused. This is visible in confusion matrix. Especialy right side of matrix.

In [60]:
X_train_tokens = vect_all.get_feature_names()

In [106]:

def counter():
    res = {'token':X_train_tokens}
    for i, c in enumerate(nb_all.classes_):
        res[str(c)] = np.delete(nb_all.feature_count_[i, :], i)
        print(res[str(c)].shape)
    return res

In [107]:
counter()

(25796,)
(25796,)
(25796,)
(25796,)
(25796,)


{'1': array([ 5.,  0.,  1., ...,  0.,  0.,  0.]),
 '2': array([ 28.,   0.,   0., ...,   0.,   0.,   0.]),
 '3': array([ 30.,   2.,   0., ...,   0.,   0.,   0.]),
 '4': array([ 49.,   3.,   1., ...,   1.,   0.,   1.]),
 '5': array([ 33.,   7.,   0., ...,   0.,   1.,   0.]),
 'token': ['00',
  '000',
  '007',
  '00a',
  '00am',
  '00pm',
  '01',
  '02',
  '04',
  '05',
  '06',
  '07',
  '08',
  '09',
  '0buxoc0crqjpvkezo3bqog',
  '0l',
  '0tzg',
  '10',
  '100',
  '1000',
  '1001',
  '100lbs',
  '100s',
  '100th',
  '101',
  '102',
  '102729',
  '1030',
  '104',
  '105',
  '1070',
  '107f',
  '108',
  '109',
  '10am',
  '10ish',
  '10k',
  '10min',
  '10mins',
  '10minutes',
  '10oz',
  '10p',
  '10pm',
  '10th',
  '10x',
  '10yo',
  '11',
  '110',
  '1100',
  '111',
  '112',
  '113',
  '1130',
  '114',
  '115',
  '116',
  '117',
  '118',
  '11a',
  '11am',
  '11p',
  '11pm',
  '11year',
  '12',
  '120',
  '1200',
  '12000',
  '1202',
  '123',
  '124',
  '128i',
  '12a',
  '12am',
  '12k

In [103]:
token_counts = pd.DataFrame(counter())

ValueError: arrays must all be same length

In [94]:
token_counts.head()

,1,2,3,4,5,token,sum
0,19,28,30,49,33,00,846849
1,5,2,2,3,7,000,846849
2,0,0,0,1,0,007,846849
3,1,0,0,0,0,00a,846849
4,3,0,0,2,2,00am,846849


In [96]:
token_counts.0

SyntaxError: invalid syntax (<ipython-input-96-27a15a8eb8f0>, line 1)

In [89]:
nb_all.feature_count_[1:]

array([[ 28.,   2.,   0., ...,   0.,   0.,   0.],
       [ 30.,   2.,   0., ...,   0.,   0.,   0.],
       [ 49.,   3.,   1., ...,   1.,   0.,   1.],
       [ 33.,   7.,   0., ...,   0.,   1.,   0.]])

In [ ]:
def compute_ratio(i, df):
    df[i, ]

In [87]:
token_counts['sum'] = np.sum(nb_all.feature_count_[1:])

In [88]:
token_counts.head()

,1,2,3,4,5,token,sum
0,19,28,30,49,33,00,846849
1,5,2,2,3,7,000,846849
2,0,0,0,1,0,007,846849
3,1,0,0,0,0,00a,846849
4,3,0,0,2,2,00am,846849


In [46]:
token_counts['one'] = token_counts.one + 1

In [63]:

nb_all.feature_count_[0, :]

array([ 19.,   5.,   0., ...,   0.,   0.,   0.])

In [58]:
nb_all.feature_count_[1:, :].shape

(4, 25797)

In [54]:
np.sum(nb_all.feature_count_[1:, :])

846849.0

In [53]:
import numpy as np

In [55]:
token_counts['one'] = token_counts.one / np.sum(nb_all.feature_count_[1:, :])

In [64]:
token_counts.sort("one", ascending=False).head()

,five,one,token
23029,5735,4310,the
1262,3182,2685,and
23317,2721,2477,to
24893,1972,1517,was
12187,1751,1346,it
